In [1]:
import tensorflow as tf
from model import *
from utils import PlotCallback, get_gp_curve_generator

# DataLoader

In [2]:
BATCH_SIZE = 32
EPOCHS = 10

In [3]:
# train_ds = tf.data.Dataset.from_generator(
#     get_gp_curve_generator(
#         iterations=250,
#         batch_size=BATCH_SIZE,
#         max_num_context=10,
#         testing=False
#     ),
#     output_signature=(
#         ((
#             tf.TensorSpec(shape=(), dtype=tf.float32),
#             tf.TensorSpec(shape=(), dtype=tf.float32),
#             tf.TensorSpec(shape=(), dtype=tf.float32)
#         ),
#         tf.TensorSpec(shape=(), dtype=tf.float32)))
# )


train_ds = tf.data.Dataset.from_generator(
    get_gp_curve_generator(
        iterations=250,
        batch_size=BATCH_SIZE,
        max_num_context=10,
        testing=False),
    output_types=((tf.float32, tf.float32, tf.float32), tf.float32)
)

test_ds = tf.data.Dataset.from_generator(
    get_gp_curve_generator(
        iterations=250,
        batch_size=1,
        testing=True),
    output_types=((tf.float32, tf.float32, tf.float32), tf.float32)
)

Metal device set to: Apple M1


2023-02-04 16:20:10.360847: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-04 16:20:10.360977: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
iterator = get_gp_curve_generator(
        iterations=250,
        batch_size=2,
        testing=True)()

In [5]:
(context_x, context_y, target_x), target_y = next(iterator)

In [6]:
context_x.shape

TensorShape([2, 9, 1])

In [7]:
context_y.shape

TensorShape([2, 9, 1])

In [8]:
target_x.shape

TensorShape([2, 400, 1])

In [9]:
target_y.shape

TensorShape([2, 400, 1])

In [10]:
train_ds

<FlatMapDataset element_spec=((TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

In [11]:
train_ds.element_spec

((TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
  TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
  TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)),
 TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))

In [12]:
tf.data.experimental.cardinality(train_ds)

<tf.Tensor: shape=(), dtype=int64, numpy=-2>

# Loss

In [13]:
import tensorflow_probability as tfp
tfd = tfp.distributions

In [57]:
# Initialize a single 2-variate Gaussian.
dist = tfd.MultivariateNormalDiag(
    loc=[1., -1],
    scale_diag=[1, 2.])

In [58]:
dist.log_prob(value=[0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=-3.1560242>

# Model

In [19]:
encoder_dims = [128, 128, 128, 128]
decoder_dims = [128, 128, 2]

In [20]:
def loss(target_y, pred_y):
    # Get the distribution
    mu, sigma = tf.split(pred_y, num_or_size_splits=2, axis=-1)
    dist = tfd.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
    return -dist.log_prob(target_y)

In [34]:
model = ConditionalNeuralProcess(encoder_dims, decoder_dims)

In [35]:
model

In [40]:
model.compile(loss=loss, optimizer='adam')

In [41]:
tf.test.gpu_device_name()

2023-02-04 16:36:19.284545: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-04 16:36:19.284580: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


'/device:GPU:0'

In [55]:
model.fit(train_ds, epochs=EPOCHS)

Epoch 1/10
250/250 [==============================] - 25s 99ms/step - loss: 0.6126
Epoch 2/10
250/250 [==============================] - 24s 98ms/step - loss: 0.5544
Epoch 3/10
250/250 [==============================] - 24s 98ms/step - loss: 0.5009
Epoch 4/10
250/250 [==============================] - 25s 100ms/step - loss: 0.4405
Epoch 5/10
250/250 [==============================] - 25s 99ms/step - loss: 0.4168
Epoch 6/10
250/250 [==============================] - 25s 99ms/step - loss: 0.3739
Epoch 7/10
250/250 [==============================] - 25s 101ms/step - loss: 0.3341
Epoch 8/10
250/250 [==============================] - 25s 100ms/step - loss: 0.3016
Epoch 9/10
250/250 [==============================] - 26s 102ms/step - loss: 0.2646
Epoch 10/10
250/250 [==============================] - 26s 104ms/step - loss: 0.2161
